# Embeddings desde cero (Capítulo 2 - LLMs from Scratch)

En este notebook implemento los conceptos clave del capítulo 2 del libro *Build a Large Language Model (From Scratch)* de Sebastian Raschka.

El objetivo es entender cómo los LLM transforman texto en vectores numéricos (embeddings), ya que estos son la base de:

- Modelos de lenguaje
- Sistemas de recuperación semántica
- Agentes inteligentes
- RAG (Retrieval-Augmented Generation)

En lugar de usar librerías de alto nivel, reproducimos el proceso manualmente para entender:

1. Tokenización
2. Ventanas deslizantes
3. Creación de datasets de entrenamiento
4. Generación de embeddings con PyTorch

In [8]:
!pip install torch tiktoken --quiet

In [9]:
import urllib.request

url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
filename = "the-verdict.txt"

urllib.request.urlretrieve(url, filename)

with open(filename, "r", encoding="utf-8") as f:
    text = f.read()

print("Longitud del texto:", len(text))
print(text[:500])

Longitud del texto: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it'


## ¿Por qué usar texto real?

Entrenar modelos con texto real permite simular cómo funcionan los LLM en producción.

El texto usado en el libro es un cuento corto que permite:

- Tener un dataset pequeño y manejable
- Ver patrones lingüísticos reales
- Experimentar con tokenización realista

Esto es importante porque los embeddings dependen directamente de la distribución del lenguaje. Un embedding entrenado con texto artificial no captura semántica real.

In [10]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

tokens = tokenizer.encode(text)
print("Número de tokens:", len(tokens))
print(tokens[:20])

Número de tokens: 5145
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438]


## ¿Por qué tokenizar es crítico?

Los LLM no entienden texto directamente. Solo procesan números.

La tokenización convierte texto en unidades discretas llamadas tokens. Estas pueden ser:

- Palabras
- Subpalabras
- Caracteres

En modelos modernos se usan subpalabras porque balancean:

- Vocabulario manejable
- Buena representación semántica

Sin tokenización no existirían embeddings ni transformers, porque las redes neuronales solo operan sobre tensores numéricos.

In [11]:
import torch

max_length = 4
stride = 1

input_ids = []
target_ids = []

for i in range(0, len(tokens) - max_length, stride):
    input_chunk = tokens[i:i + max_length]
    target_chunk = tokens[i + 1:i + max_length + 1]

    input_ids.append(input_chunk)
    target_ids.append(target_chunk)

print("Número de muestras:", len(input_ids))
print("Ejemplo input:", input_ids[0])
print("Ejemplo target:", target_ids[0])

Número de muestras: 5141
Ejemplo input: [40, 367, 2885, 1464]
Ejemplo target: [367, 2885, 1464, 1807]


## Ventanas deslizantes en modelos de lenguaje

Los LLM se entrenan prediciendo el siguiente token. Para ello se crean ventanas deslizantes.

Ejemplo:
Input:  [A, B, C, D]  
Target: [B, C, D, E]

Esto enseña al modelo dependencias secuenciales.

El stride controla cuánto se mueve la ventana:
- stride pequeño → más datos, más solapamiento
- stride grande → menos datos, menos contexto compartido

Este mecanismo es clave en:
- Transformers
- Modelos autoregresivos
- Sistemas de generación de texto

In [12]:
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, input_ids, target_ids):
        self.input_ids = input_ids
        self.target_ids = target_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx]), torch.tensor(self.target_ids[idx])

dataset = GPTDataset(input_ids, target_ids)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

for x, y in dataloader:
    print("Batch input shape:", x.shape)
    break

Batch input shape: torch.Size([8, 4])


## Importancia de estructurar datasets

Convertir datos en un Dataset de PyTorch permite:

- Entrenamiento eficiente en GPU
- Batching automático
- Paralelismo

Esto es exactamente lo que hacen frameworks como:
- HuggingFace
- DeepSpeed
- SageMaker Training Jobs

Entender esta capa es clave para construir agentes propios o entrenar modelos personalizados.

In [13]:
vocab_size = tokenizer.n_vocab
embedding_dim = 256

embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)

sample_input = torch.tensor(input_ids[:2])
embedded = embedding_layer(sample_input)

print("Shape embeddings:", embedded.shape)

Shape embeddings: torch.Size([2, 4, 256])


## ¿Qué son embeddings?

Un embedding es una representación vectorial densa de un token.

En lugar de usar one-hot encoding (vectores enormes y dispersos), los embeddings:

- Reducen dimensionalidad
- Capturan relaciones semánticas
- Permiten generalización

Ejemplo conceptual:
rey - hombre + mujer ≈ reina

Esto emerge porque los embeddings aprenden patrones estadísticos del lenguaje.

## ¿Por qué los embeddings codifican significado y cómo se relacionan con redes neuronales?

Los embeddings codifican significado porque se entrenan mediante optimización de redes neuronales profundas.

Durante el entrenamiento:
1. El modelo predice el siguiente token
2. Calcula error (loss)
3. Backpropagation ajusta pesos

La capa de embedding es simplemente una matriz entrenable. Cada fila corresponde a un token.

Con el tiempo:
- Tokens usados en contextos similares convergen en regiones cercanas del espacio vectorial
- Esto crea geometría semántica

Relación con conceptos de NN:
- Es una capa lineal entrenable
- Aprende representaciones latentes
- Funciona como reducción de dimensionalidad supervisada

Por eso los embeddings son la base de:
- Búsqueda semántica
- RAG
- Agentes LLM
- Sistemas multimodales

In [14]:
def count_samples(max_length, stride):
    count = 0
    for i in range(0, len(tokens) - max_length, stride):
        count += 1
    return count

configs = [
    (4, 1),
    (4, 2),
    (8, 1),
    (8, 4),
]

for max_len, stride_val in configs:
    samples = count_samples(max_len, stride_val)
    print(f"max_length={max_len}, stride={stride_val} → muestras={samples}")

max_length=4, stride=1 → muestras=5141
max_length=4, stride=2 → muestras=2571
max_length=8, stride=1 → muestras=5137
max_length=8, stride=4 → muestras=1285


## Experimento: efecto de max_length y stride

Resultados observados:
- Reducir stride aumenta el número de muestras
- Aumentar max_length reduce el número de ventanas posibles

¿Por qué ocurre esto?

Número de muestras ≈ (N - max_length) / stride

### Interpretación

Stride pequeño:
- Más datos de entrenamiento
- Más solapamiento
- Mejor aprendizaje contextual

Stride grande:
- Menos redundancia
- Entrenamiento más rápido
- Menor captura de dependencias largas

### ¿Por qué el solapamiento es útil?

El solapamiento permite que el modelo vea el mismo token en múltiples contextos.

Esto mejora:
- Robustez semántica
- Generalización
- Calidad de embeddings

Este principio también se usa en:
- Sliding window attention
- Chunking en RAG
- Procesamiento de documentos largos

## Conclusiones

En este laboratorio implementé desde cero el pipeline base de embeddings:

- Tokenización
- Ventanas deslizantes
- Dataset en PyTorch
- Capa de embeddings

Esto demuestra que los embeddings no son magia, sino matrices entrenables que emergen de optimización neuronal.

Entender este proceso es fundamental para:
- Construir LLMs propios
- Diseñar sistemas RAG
- Desarrollar agentes autónomos
- Optimizar despliegues en SageMaker o producción

Este conocimiento conecta teoría y práctica, permitiendo pasar de usuario de LLMs a constructor de modelos.